In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from hep_ml.uboost import uBoostClassifier
import sys
sys.path.append('../fairboost')

from generate import generate_toys
from generate import show_variates
from plot import show_clf

# Show the data

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
show_variates(ax, generate_toys)


# Train a classifier

## normal bdt

In [ ]:
X, Y, Z = generate_toys(10000)

clf = GradientBoostingClassifier(n_estimators=100, verbose=True)
clf.fit(X, Y)

show_clf(clf, generate_toys)

## uniform boosting

X, Y, Z = generate_toys(100, pandas=True)

clf = uBoostClassifier(n_estimators=100, uniform_features=['z'], uniform_label=1, train_features=['x1', 'x2'])
clf.fit(X, Y)

show_clf(clf, generate_toys, pandas=True)

# Custom classifier

In [ ]:
from scipy.special import expit
from scipy.special import logit
from sklearn.ensemble._gb_losses import BinomialDeviance
from sklearn.metrics import roc_auc_score
from models import PolynomialModel

import warnings
warnings.filterwarnings("ignore")

def debug(x, f, z, Y, derivative, adv):
    
    # create model response
    xs = np.linspace(0,1,100)
    ys = adv.predict(xs)
    
    # create bins
    nbins=100
    hist, edges = np.histogram(f, range=(0,1), bins=nbins)
    centres = 0.5 * (edges[:-1] + edges[1:])

    # compute metrics over bins
    idx = np.digitize(f, edges)
    bin_means = [np.mean(z[idx==i]) for i in range(1, len(edges))]
    bin_stds = [np.std(z[idx==i])/np.sqrt(len(z[idx==i])) for i in range(1, len(edges))]
    
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    
    # show Z as a function of F
    ax[0].scatter(f[Y==1], z[Y==1], alpha=0.1, color='darkblue')
    ax[0].errorbar(centres, bin_means, xerr=0.5/nbins, yerr=bin_stds, color='lightblue')
    ax[0].plot(xs, ys, color='red')
    ax[0].set_xlabel('classifier output')
    ax[0].set_ylabel('Z')
    ax[0].set_xlim(0,1)
    
    # show variates
    ax[1].scatter(X[Y==1,0], X[Y==1,1], alpha=0.5, c=derivative[Y==1], cmap='coolwarm')
    ax[1].set_title('derivative')
    ax[1].set_xlabel('x0')
    ax[1].set_ylabel('x1')
    ax[1].set_xlim(-1, 2)
    ax[1].set_ylim(-1, 3)
    plt.show()

class FairboostClassifier:
    
    def __init__(self, Z, n_estimators=100, learning_rate=0.1, max_depth=3):
        
        self.Z = Z
        self.lam = 1.0
        self.order = 1
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        
        self._loss = BinomialDeviance(2)
        self._estimators = []
        
    def _raw_prediction(self, X):
        
        # initial model TODO: this is loss function dependent
        raw_prediction = logit(self.prior) * np.ones(len(X))
        
        # loop over estimators
        for i, est in enumerate(self._estimators):
            
            # estimator response
            pred = est.predict(X)
            
            # add them up
            raw_prediction += self.learning_rate * pred
        
        return raw_prediction.reshape(-1,1)
    
    def fit(self, X, Y):
        
        # force type
        X = np.array(X, dtype=np.float32)
        
        # initial model
        self.prior = np.sum(Y==1) / len(Y)
        
        # fit the remaining estimators
        for i in range(self.n_estimators):
            
            # predictions from the previous models
            raw_prediction = self._raw_prediction(X)
            raw_prediction_copy = raw_prediction.copy() # see sklearn documentation
            
            # compute the gradient
            neg_grad = self._loss.negative_gradient(Y, raw_prediction_copy)
            
            #########################
            # the adversarial part
            #########################
            
            # give it a bit of time
            if i>9:
                f = self.predict_proba(X)[:,1]
                
                dLdf = get_gradient_vanilla(f, self.Z, self.order, i, self.n_estimators)
                
                neg_grad += - self.lam * dLdf
            
            #########################
            #########################
            
            # fit the new tree
            tree = DecisionTreeRegressor(max_depth=self.max_depth,
                                         criterion='friedman_mse')
            tree.fit(X, neg_grad)
            
            # line search for each leaf (done in the loss function method)
            sample_weight = np.ones(shape=Y.shape)
            sample_mask = np.ones(shape=Y.shape, dtype=bool)
            
            # the following part makes implementation equivalent to sklearn but messes up the adversarial part
            
            #self._loss.update_terminal_regions(
            #    tree.tree_, X, Y, neg_grad, raw_prediction,
            #    sample_weight, sample_mask, self.learning_rate)
            
            # append results
            self._estimators.append(tree)
        
    def predict_proba(self, X):
        
        # make raw predictions
        raw_prediction = self._raw_prediction(X)
        
        # turn them into a probability
        proba = self._loss._raw_prediction_to_proba(raw_prediction)
        
        return proba

def get_gradient_vanilla(f, z, order, i, n_est):
    """
    Mean square loss as the adversary loss.
    Polynomial model of the Z as a function of f.
    """
    # compute the adversary gradient: dL/df = dL/dA * dA/df
    adv = PolynomialModel(order)
    adv.fit(f, z)
    A = adv.predict(f)
                
    dLdA = (A - z)
    dAdf = adv.gradient(f)
    dLdf = dLdA * dAdf # dL/df. in my understanding should have a negative sign in front but then it doesnt work empirically
                
    if i%10==0:
        print('{}/{}'.format(i, n_est))
        debug(X, f, z, Y, dLdf, adv)
            
    # combine to the total gradient
    return dLdf
    
def compare_classifiers(N, generate):

    # training and test sets
    X, Y, Z = generate(N)
    X_test, Y_test, Z_test = generate(N)

    # fit the models
    fb_clf = FairboostClassifier(Z, n_estimators=100)
    sk_clf = GradientBoostingClassifier(n_estimators=100)
    fb_clf.fit(X, Y)
    sk_clf.fit(X, Y)

    # predict
    fb_proba = fb_clf.predict_proba(X_test)[:,1]
    sk_proba = sk_clf.predict_proba(X_test)[:,1]

    # evaluate
    fb_score = roc_auc_score(Y_test, fb_proba)
    sk_score = roc_auc_score(Y_test, sk_proba)

    print('Fairboost classifier:', fb_score)
    show_clf(fb_clf, generate)
    print('Sklearn classifier:', sk_score)
    show_clf(sk_clf, generate)

compare_classifiers(10000, generate_toys)


In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
show_variates(ax, generate_toys)

